# Subduction Zone Benchmark

Author: Cian Wilson

## Parallel Scaling

In [the previous notebook](./2.5b_blankenbach.ipynb) we tested that the error in our implementation of a steady-state thermal convection problem in two-dimensions converged towards the published benchmark value as the number of elements increased.  We also wish to test for parallel scaling of this problem, assessing if the simulation wall time decreases as the number of processors used to solve it is increases.

Here we perform strong scaling tests on our function `solve_blankenbach` from [`notebooks/02_background/2.5b_blankenbach.ipynb`](./2.5b_blankenbach.ipynb).  We will see that as we perform multiple solves in the Picard iteration the advantages of an iterative solver are eclipsed for this simple 2D case as the direct solver can reuse its initial expensive analysis step on subsequent solves.  This also helps to overcome some of the poor scaling we observed using the direct solver in previous examples.

### Preamble

We start by loading all the modules we will require.

In [ ]:
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
path = os.path.join(basedir, os.path.pardir, os.path.pardir, 'python')
sys.path.append(path)
import utils.ipp
import matplotlib.pyplot as pl
import numpy as np
import pathlib
output_folder = pathlib.Path(os.path.join(basedir, "output"))
output_folder.mkdir(exist_ok=True, parents=True)

### Implementation

We perform the strong parallel scaling test using a utility function (from [`python/utils/ipp.py`](../../python/utils/ipp.py)) that loops over a list of the number of processors calling our function for a given number of elements, `ne`, and pressure and temperature polynomial orders `pp` and `pT`.  It runs our function `solve_blankenbach` a specified `number` of times and evaluates and returns the time taken for each of a number of requested `steps`.

In [ ]:
# the list of the number of processors we will use
nprocs_scale = [1, 2,]

# the number of elements to solve the problem on
resscale = 1

# perform the calculation a set number of times
number = 10

# We are interested in the time to create the mesh,
# declare the functions, assemble the problem and solve it.
# From our implementation in `solve_poisson_2d` it is also
# possible to request the time to declare the Dirichlet and
# Neumann boundary conditions and the forms.
steps = [
          'Assemble Temperature', 'Assemble Stokes',
          'Solve Temperature', 'Solve Stokes'
         ]


#### Case 1

In [ ]:
# declare a dictionary to store the times each step takes
maxtimes_1 = {}

In [ ]:
maxtimes_1['Direct Stokes'] = utils.ipp.profile_parallel(nprocs_scale, steps, path, 
                                                         'sz_problems.sz_benchmark', 'solve_benchmark_case1', 
                                                        resscale, number=number,
                                                        output_filename=output_folder / 'sz_benchmark_scaling_direct_1a.png')

In [ ]:
# the list of the number of processors to test the convergence on
nprocs_conv = [1, 2,]

# List of resolutions to try
resscales = [4, 2,]

In [ ]:
errors_1 = []
for resscale in resscales:
    errors_1.append(utils.ipp.run_parallel(nprocs_conv, path, 
                                           'sz_problems.sz_benchmark', 
                                           'solve_benchmark_case1', 
                                           resscale))

print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('nprocs', 'resscale', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
for resscale, diag_all in zip(resscales, errors_1):
    for i, nproc in enumerate(nprocs_conv):
        print('{:<12d} {:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(nproc, resscale, *diag_all[i].values()))